Copyright Vespa.ai. Licensed under the terms of the Apache 2.0 license. See LICENSE in the project root.


# ES


## Running the Elasticsearch docker container


```bash
docker run -p 9200:9200 -d --name elasticsearch --rm \
  -e "discovery.type=single-node" \
  -e "xpack.security.enabled=false" \
  -e "xpack.security.http.ssl.enabled=false" \
  -e "xpack.license.self_generated.type=trial" \
  docker.elastic.co/elasticsearch/elasticsearch:8.13.2
```


### Verify that docker/podman engine is running

Recommended resource configuration:

- 8 CPUs
- 24 GB RAM
- 100+ GB disk space


In [ ]:
!docker info

In [ ]:
import docker

client = docker.from_env()
docker_network = "my_network"

# Create a network if it does not exist
try:
    client.networks.get(docker_network)
    print(f"Network {docker_network} already exists")
except docker.errors.NotFound:
    client.networks.create(docker_network, driver="bridge")
    print(f"Network {docker_network} created")

container = client.containers.run(
    "docker.elastic.co/elasticsearch/elasticsearch:8.13.2",
    detach=True,
    ports={"9200/tcp": 9200},
    network=docker_network,
    name="elasticsearch",
    environment=[
        "discovery.type=single-node",
        "xpack.security.enabled=false",
        "xpack.security.http.ssl.enabled=false",
        "xpack.license.self_generated.type=trial",
    ],
)
# Wait until container is ready
container.reload()

In [ ]:
from elasticsearch import Elasticsearch
import json
import time

es = Elasticsearch("http://localhost:9200")
timeout = 30
while not es.ping():
    timeout -= 1
    if timeout == 0:
        raise TimeoutError("Elasticsearch is not ready")
    time.sleep(1)

In [ ]:
index_name = "product"

# Delete the index if it exists
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

In [ ]:
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 1},
    "mappings": {
        "properties": {
            "id": {"type": "integer"},
            "title": {"type": "text", "similarity": "BM25"},
            "description": {"type": "text", "similarity": "BM25"},
            "category": {"type": "keyword"},
            "price": {"type": "integer"},
            "average_rating": {"type": "float"},
            "embedding": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine",
                # See https://www.elastic.co/guide/en/elasticsearch/reference/current/dense-vector.html
                "index_options": {"type": "hnsw", "ef_construction": 200, "m": 16},
            },
        },
    },
}

# Create the index
es.indices.create(index=index_name, body=index_settings)

In [ ]:
# Pretty-print the mapping
mapping = es.indices.get_mapping(index=index_name)
mapping.raw

## Feed the data to the Elasticsearch container


In [ ]:
!sh feed_to_es.sh

In [ ]:
es.indices.refresh(index=index_name)
refresh_result = es.cat.count(index=index_name, params={"format": "json"})
refresh_result

In [ ]:
assert refresh_result[0]["count"] == "1000000"

For single file (<100MB):

```bash
curl -s -H "Content-Type: application/x-ndjson" -XPOST localhost:9200/_bulk --data-binary "@../dataprep/output-data/final/es_feed-10k.json"
```


## Running fbench against ES-container


Decompressing query-files


In [ ]:
!for f in ../dataprep/output-data/final/es_queries-*-10k.json.zst; do zstd -d -f "$f" -o "${f%.zst}"; done

### Docker command to run fbench against ES-container


```bash
docker run -v /Users/thomas/Repos/system-test/tests/performance/ecommerce_hybrid_search/dataprep/output-data/final:/files -w /files \
--network my_network \
--entrypoint /opt/vespa/bin/vespa-fbench \
vespaengine/vespa \
 -c 0 -s 5 -n 1 -q es_queries-weak_and-10k.json -P -o fbench_output_es_weak_and.txt -D elasticsearch 9200
```


In [ ]:
# Define the options and base filenames
options = ["weak_and", "semantic", "hybrid"]
base_query_file = "es_queries-{}-10k.json"
base_output_file = "fbench_output_{}.txt"
result_file = "fbench_results_{}.txt"
# Generate the configurations dynamically
configs = [
    {
        "option": option,
        "query_file": base_query_file.format(option),
        "output_file": base_output_file.format(option),
        "result_file": result_file.format(option),
    }
    for option in options
]

# Loop through each configuration and run the container
for config in configs:
    print(f"Running fbench in container for {config['option']} queries")
    output = client.containers.run(
        image="vespaengine/vespa",
        entrypoint="/opt/vespa/bin/vespa-fbench",  # Set vespa-fbench as the entrypoint
        network=docker_network,
        command=[
            "-c",
            "0",
            "-s",
            "30",
            "-n",
            "1",
            "-q",
            config["query_file"],
            "-P",
            "-o",
            config["output_file"],
            "-D",
            "elasticsearch",
            "9200",
        ],
        volumes={
            "/Users/thomas/Repos/system-test/tests/performance/ecommerce_hybrid_search/dataprep/output-data/final": {
                "bind": "/files",
                "mode": "rw",
            }
        },
        working_dir="/files",
        detach=False,
        remove=True,
    )

    # Wait for the container to finish and print the output
    result = output.decode("utf-8")
    print(f"Output for {config['option']} queries:\n{result}")
    # Save results to a file
    with open(config["result_file"], "w") as file:
        file.write(result)

In [ ]:
# Stop the ES container
container.stop()
container.remove()

In [ ]:
assert False  # Just to stop the execution here